In [14]:
!pip install tqdm requests

In [16]:
import os
import requests
from tqdm.notebook import tqdm
from pathlib import Path

In [18]:
def create_folder(path):
    Path(path).mkdir(parents=True, exist_ok=True)

def download_image(url, path):
    try:
        response = requests.get(url, timeout=10)
        if response.status_code == 200:
            with open(path, 'wb') as f:
                f.write(response.content)
            return True
    except Exception as e:
        pass
    return False

In [19]:
def get_pokemon_data(start: int, end: int):
    url = "https://pokeapi.co/api/v2/pokemon?limit=10000"
    response = requests.get(url)
    data = response.json()
    
    pokemon = []
    for entry in data['results']:
        name = entry['name']
        try:
            url_id = int(entry['url'].split('/')[-2])
        except:
            continue
        if start <= url_id <= end:
            pokemon.append((name, url_id))
    return pokemon


In [23]:
def baixar_pokemon(pokemon_name: str, pokemon_id: int, folder_path: str):
    forms = ["", "-mega", "-megax", "-megay", "-gmax", "-alola", "-galar", "-hisui", "-paldea"]
    
    # Formatar o nome da pasta como "0006 - Charizard"
    folder_name = f"{str(pokemon_id).zfill(4)} - {pokemon_name.capitalize()}"
    folder = Path(folder_path) / folder_name
    folder.mkdir(parents=True, exist_ok=True)
    
    base_name = pokemon_name.lower()
    
    for form in forms:
        form_name = base_name + form
        
        # PNGs (oficial e shiny)
        if form == "":
            # Oficial
            png_url = f"https://assets.pokemon.com/assets/cms2/img/pokedex/full/{str(pokemon_id).zfill(3)}.png"
            download_image(png_url, folder / f"{form_name}.png")
        
        else:
            # Sprites alternativos que não existem no site oficial
            static_url = f"https://play.pokemonshowdown.com/sprites/ani/{form_name}.gif".replace(".gif", ".png")
            shiny_static_url = f"https://play.pokemonshowdown.com/sprites/ani-shiny/{form_name}.gif".replace(".gif", ".png")
            download_image(static_url, folder / f"{form_name}.png")
            download_image(shiny_static_url, folder / f"{form_name}-shiny.png")
        
        # GIFs (normais e shiny)
        gif_url = f"https://play.pokemonshowdown.com/sprites/ani/{form_name}.gif"
        shiny_gif_url = f"https://play.pokemonshowdown.com/sprites/ani-shiny/{form_name}.gif"
        download_image(gif_url, folder / f"{form_name}.gif")
        download_image(shiny_gif_url, folder / f"{form_name}-shiny.gif")


In [26]:
# Caminho de destino
base_folder = r"C:\Users\corps\Documents\Cursor\pokedex\jupyter\images"
create_folder(base_folder)

# Faixa da Pokédex
start_id = int(input("Número inicial da Pokédex: "))
end_id = int(input("Número final da Pokédex: "))

# Obter lista
lista_pokemon = get_pokemon_data(start_id, end_id)

# Baixar todos
for name, pid in tqdm(lista_pokemon):
    baixar_pokemon(name, pid, base_folder)


Número inicial da Pokédex: 21
Número final da Pokédex: 50


  0%|          | 0/30 [00:00<?, ?it/s]